# 决策树
决策树(decision tree)是一种基本的分类与回归算法。在分类问题中，表示基于特征对样本进行分类的过程，可以认为它是一种**if-then**规则的集合。决策树模型的学习过程通常包括三个步骤：
- 特征选择
- 决策树的生成
- 决策树的修剪

## 1.模型定义
决策树模型是一种由**节点(node)** 和 **有向边(directed edge)** 组成的树形结构。其中节点有两种类型：
- **内部节点(internal node)：**表示某一个特征
- **叶节点(leaf node)：**表示某个类  
  
决策树进行分类的过程：从根节点开始，对样本的某一特征进行判断，根据判断结果将样本分配到子节点，然后如此递归的进行判断并分配，直至到达某一叶节点，则该样本属于该叶节点所表示的类。
> **每个样本从根节点到叶节点，有且只有一条路径。**  

![决策树模型](./images/dt.png)
上图是一个简单的决策树模型，根据某人“是否有房产”“是否结婚”以及“月收入”这三个特征，来判断能否偿还贷款。

## 2.特征选择
在生成决策树的过程中涉及到了递归选择**最优特征**的过程，这个过程至关重要，因为它直接决定了模型的生成效率和性能。  
  
怎样的特征才是最优的特征呢？  

直观上，当训练集被某个特征划分成子集后，使得各个子集中的样本**不确定性越小**，也就是说，使得各个子集中的样本类别越统一(最好的情况是所有样本属于同一类)，那么这个特征就是最优特征。  

在介绍如何选择最优特征之前，先来介绍两个重要的基本概念：熵和条件熵。

### 2.1 熵
在信息论和概率统计中，**熵(entropy)** 是用来表示随机变量不确定性的指标。  
  
设$X$是一个离散随机变量，其概率分布为：$$P(X=x_{i})=p_{i},\quad i=1,2,...,n$$
则随机变量$X$的熵定义为：$$H(X)=-\sum_{i=1}^{n}p_{i}\log p_{i}$$ 
上式中的对数通常以2为底或以e为底，若$p_{i}=0$，则$0log0=0$。由定义可知，熵只与$X$的分布有关，所以可以将$H(X)$记作$H(p)$，且有$0\leq H(p)\leq logn$。 

> **熵越大，随机变量的不确定性就越大。**   
  
假设随机变量$X$只取0,1两个值，则概率分布为：$P(X=1)=p， P(X=0)=1-p， 0\leq p \leq 1$  
熵为：$H(p)=-plog_{2}p-(1-p)log_{2}(1-p)$  
熵$H(p)$与概率$p$之间的关系如下图所示：![entropy.png](./images/entropy.png)  
当$p=0$或$p=1$时$H(p)=0$，即当随机变量全是0或者全是1时，完全没有不确定性；当$p=0.5$时$H(p)=1$，即当随机变量取各个值的概率都相等时，不确定性最大。

### 2.2 条件熵
设随机变量$(X,Y)$的联合概率分布为：$$P(X=x_{i},Y=y_{j})=p_{ij}，i=1,2,...,n；j=1,2,...,m$$  
则在随机变量$X$给定的条件下随机变量$Y$的**条件熵(conditional entropy)** 定义为：$$H(Y\,|\,X)=\sum_{i=1}^{n}p_{i}H(Y\,|\,X=x_{i})$$  
其中，$p_{i}=P(X=x_{i}),\,i=1,2,...,n$  
> **条件熵$H(Y\,|\,X)$表示在已知随机变量$X$的条件下随机变量$Y$的不确定性。**

### 2.3 信息增益
现在，知道了熵和条件熵后，就可以基于**信息增益(information gain)** 准则来选择最优的特征了。  
  
**特征 $A$ 对数据集 $D$ 的信息增益 $g(D,A)$ 定义为：$$g(D,A)=H(D)-H(D\,|\,A)$$
$H(D)$ 表示数据集 $D$ 中的类别不确定性程度，$H(D\,|\,A)$ 表示在给定特征 $A$ 的条件下数据集 $D$ 的类别不确定性程度，它们的差则表示由于特征 $A$ 而使得数据集 $D$ 的类别不确定性减少的程度。**一般情况下，不同的特征具有不同的信息增益，信息增益最大的特征就是应该选择的最优特征。  
  
设训练集为 $D$，$|\,D\,|$ 表示训练集中的样本个数，总共有 $K$ 个类 $C_{k},\,k=1,2,...,K$，$|\,C_{k}\,|$ 表示每个类别的样本个数，$\sum_{k=1}^{K}|\,C_{k}\,|=|\,D\,|$。设总共有 $M$ 个特征 $\{A_{1},A_{2},...,A_{M}\}$，对于其中某个特征 $A_{m}$，假设有 $n$ 个不同的取值 $\{a_{1},a_{2},...,a_{n}\}$，根据特征 $A_{m}$ 的取值将 $D$ 划分为 $n$ 个子集 $\{D_{1},D_{2},...,D_{n}\}$，$|\,D_{i}\,|$ 表示子集 $D_{i}$ 的样本个数，$\sum_{i=1}^{n}|\,D_{i}|=|\,D\,|$。设子集 $D_{i}$ 中属于类 $C_{k}$ 的样本集合为 $D_{ik}$，即 $D_{ik}=D_{i}\cap C_{k}$，$|\,D_{ik}\,|$ 表示 $D_{ik}$ 的样本个数。则选择最优特征的步骤如下：  
1. 计算训练集 $D$ 的熵 $H(D)$：$$H(D)=-\sum_{k=1}^{K}\frac{|\,C_{k}\,|}{|\,D\,|}log_{2}\frac{|\,C_{k}\,|}{|\,D\,|}$$
2. 计算特征 $A_{m}$ 对训练集 $D$ 的条件熵： $H(D\,|\,A_{m})$：$$H(D\,|\,A_{m})=\sum_{i=1}^{n}\frac{|\,D_{i}\,|}{|\,D\,|}H(D_{i})=-\sum_{i=1}^{n}\frac{|\,D_{i}\,|}{|\,D\,|}\sum_{k=1}^{K}\frac{|\,D_{ik}\,|}{|\,D_{i}\,|}log_{2}\frac{|\,D_{ik}\,|}{|\,D_{i}\,|}$$ 
3. 计算信息增益：$$g(D,A_{m})=H(D)-H(D\,|\,A_{m})$$ 
4. 重复步骤2和3，算出特征集中所有的特征对应的信息增益，最后选择信息增益最大的特征作为最优特征。

### 2.4 信息增益比
信息增益存在一个不足，就是它偏向于选择取值较多的特征，为了改善这个不足，**信息增益比(information gain ratio)** 被提了出来，特征 $A$ 对数据集 $D$ 的信息增益比 $g_{R}(D,A)$ 定义为：$$g_{R}(D,A)=\frac{g(D,A)}{H_{A}(D)}$$  
其中 $g(D,A)$ 是特征 $A$ 的信息增益，$H_{A}(D)$ 是数据集 $D$ 关于特征 $A$ 的熵，$H_{A}(D)=-\sum_{i=1}^{n}\frac{|\,D_{i}\,|}{|\,D\,|}log_{2}\frac{|\,D_{i}\,|}{|\,D\,|}$，$n$ 是特征 $A$ 取值的个数。  
由上式可以看出，信息增益比会对取值较多的特征进行惩罚。

In [7]:
import pandas as pd
import numpy as np

In [13]:
def compute_entropy(labels):
    """
    计算熵
    """
    counts = np.unique(labels, return_counts=True)[1]  # |D_k|
    probs = counts / labels.size  # |D_k| / |D|
    entropy = np.round(-np.sum(probs * np.log2(probs)), 3)  # H(D)
    return entropy


def compute_cond_entropy(feature, labels):
    """
    计算条件熵
    """
    data_size = labels.size  # 数据集大小
    values = np.unique(feature)  # 获取该特征的不同取值
    subsets = [(feature[np.where(feature==v)], labels[np.where(feature==v)]) for v in values]  # 获取每个取值对应的子集, 即D_i
    probs = np.array([len(subset[0]) / data_size for subset in subsets])  # |D_i| / |D|
    subsets_entropy = np.array([compute_entropy(subset[1]) for subset in subsets])  # 计算每个子集的熵, 即H(D_i)
    cond_entropy = np.round(np.sum(probs * subsets_entropy), 3)
    return cond_entropy


def compute_gain(features, feature_names, labels):
    """
    计算每个特征的信息增益，返回最大的增益值及其对应的特征
    """
    # 计算整个数据集的熵
    H_D = compute_entropy(labels)  
    # 计算每个特征的条件熵
    cond_entropies = [compute_cond_entropy(feature, labels) for feature in features]
    # 计算信息增益
    gains = H_D - np.array(cond_entropies)
   
    # 获取最大的增益值及其对应的特征
    max_gain = np.round(np.max(gains), 3)
    feature_name = feature_names[np.argmax(gains)]
    
    return max_gain, feature_name

In [9]:
df = pd.read_csv('datas/form5_1.csv')
df

,ID,年龄,有工作,有自己的房子,信贷情况,类别
0,1,青年,否,否,一般,否
1,2,青年,否,否,好,否
2,3,青年,是,否,好,是
3,4,青年,是,是,一般,是
4,5,青年,否,否,一般,否
5,6,中年,否,否,一般,否
6,7,中年,否,否,好,否
7,8,中年,是,是,好,是
8,9,中年,否,是,非常好,是
9,10,中年,否,是,非常好,是


In [121]:
# 获取特征集, 去除'ID'列和'类别'列
feature_names = df.columns[1: -1]
features = df[feature_names].values.transpose()
# 获取类别
labels = df['类别'].values

max_gain, feature_name = compute_gain(features, feature_names, labels)
print('最优划分特征:', feature_name)
print('信息增益:', max_gain)

最优划分特征: 有自己的房子
信息增益: 0.42


## 3.决策树的生成
### 3.1 ID3
ID3算法使用**信息增益**来选择特征，递归的构建决策树。
  
具体思想：从根节点开始，计算所有可用特征的信息增益，选择具有最大信息增益的特征作为该节点的划分特征，并由该特征的不同取值建立子节点，然后对各个子节点递归地使用以上方法，直到信息增益很小或者没有特征可以选择为止，最后就得到了一个决策树。
  
> 算法流程：  
设训练集为$D$，特征集为$A$，阈值为$\epsilon$，构建决策树 $T$  
>> **Step 1:** 若$D$中的样本都属于同一类$C_{k}$，则将$C_{k}$作为该节点的类标记，返回$T$；   
**Step 2:** 若$A=\emptyset$，则将$D$中占比最大的类$C_{k}$作为该节点的类标记，返回$T$；  
**Step 3:**计算$A$中各个特征的对$D$的信息增益，选择信息增益最大的特征$A_{g}$，并做如下判断：
- 如果$A_{g}$的信息增益小于阈值$\epsilon$，则将$D$中占比最大的类$C_{k}$作为该节点的类标记，返回$T$；
- 否则，根据$A_{g}$的取值将$D$划分为若干个非空子集$D_{i}$，构建子节点，由当前节点及其子节点构成树$T$，返回$T$；  

>>**Step 4:**对第$i$个子节点，以$D_{i}$为训练集，$A-\{A_{g}\}$为特征集，递归地调用Step1到3，得到子树$T_{i}$，返回$T_{i}$。


In [138]:
# 定义树节点
class Node(object):
    def __init__(self, feature=None, label=None):
        self.feature = feature
#         self.feature_idx = -1
        self.label = label
        self.children = None


class ID3(object):
    def __init__(self, epsilon, max_depth):
        self.epsilon = epsilon
        self.max_depth = max_depth
        self.root = Node()
    
    def fit(self, X, y, feature_names):
        self.root = self.build_tree(X, y, feature_names)
        return self.root
    
    def build_tree(self, X, y, feature_names):
        features, labels = X.transpose(), y
        node = Node()
        
        unique, counts = np.unique(labels, return_counts=True)
        most_label = unique[np.argmax(counts)]  # 占比最多的类别
        # Step1 所有样本属于同一类 
        if unique.size == 1 or len(features) == 0:
            node.label = most_label
            return node
        
        # Step2 特征集为空
        if len(features) == 0:
            node.label = most_label
            return node
        
        # Step3 
        max_gain, feature_name = compute_gain(features, feature_names, labels)
        if max_gain < self.epsilon:
            node.label = most_label
            return node
        else:
            idx, other_idx = np.where(feature_names == feature_name)[0][0], np.where(feature_names != feature_name)[0]
#             node.feature_idx = idx
            feature = features[idx]
            X = np.delete(X, idx, axis=1)
            subsets_dict = {v: (X[np.where(feature == v)], y[np.where(feature == v)]) for v in np.unique(feature)}
            feature_names = feature_names[other_idx]
            node.feature = feature_name
            node.children = {key: self.build_tree(*subset, feature_names) for key, subset in subsets_dict.items()}
            return node
            

In [139]:
X = df[feature_names].values
y = df['类别'].values

id3 = ID3(0.001, -1)
tree = id3.fit(X, y, feature_names)

In [146]:
tree.children['是'].label

'是'

In [122]:
idx, other_idx = np.where(feature_names == feature_name)[0][0], np.where(feature_names != feature_name)[0]
idx, other_idx

(2, array([0, 1, 3], dtype=int64))

In [123]:
feature_names[idx], feature_names[other_idx]

('有自己的房子', Index(['年龄', '有工作', '信贷情况'], dtype='object'))

In [124]:
feature = features[idx]
feature

array(['否', '否', '否', '是', '否', '否', '否', '是', '是', '是', '是', '是', '否',
       '否', '否'], dtype=object)

In [125]:
X = df[feature_names].values
X = np.delete(X, idx, axis=1)
X

array([['青年', '否', '一般'],
       ['青年', '否', '好'],
       ['青年', '是', '好'],
       ['青年', '是', '一般'],
       ['青年', '否', '一般'],
       ['中年', '否', '一般'],
       ['中年', '否', '好'],
       ['中年', '是', '好'],
       ['中年', '否', '非常好'],
       ['中年', '否', '非常好'],
       ['老年', '否', '非常好'],
       ['老年', '否', '好'],
       ['老年', '是', '好'],
       ['老年', '是', '非常好'],
       ['老年', '否', '一般']], dtype=object)

In [131]:
d = {v: X[np.where(feature == v)] for v in np.unique(feature)}
d

{'否': array([['青年', '否', '一般'],
        ['青年', '否', '好'],
        ['青年', '是', '好'],
        ['青年', '否', '一般'],
        ['中年', '否', '一般'],
        ['中年', '否', '好'],
        ['老年', '是', '好'],
        ['老年', '是', '非常好'],
        ['老年', '否', '一般']], dtype=object), '是': array([['青年', '是', '一般'],
        ['中年', '是', '好'],
        ['中年', '否', '非常好'],
        ['中年', '否', '非常好'],
        ['老年', '否', '非常好'],
        ['老年', '否', '好']], dtype=object)}

In [134]:
for k, v in d.items():
    print(k, v)

否 [['青年' '否' '一般']
 ['青年' '否' '好']
 ['青年' '是' '好']
 ['青年' '否' '一般']
 ['中年' '否' '一般']
 ['中年' '否' '好']
 ['老年' '是' '好']
 ['老年' '是' '非常好']
 ['老年' '否' '一般']]
是 [['青年' '是' '一般']
 ['中年' '是' '好']
 ['中年' '否' '非常好']
 ['中年' '否' '非常好']
 ['老年' '否' '非常好']
 ['老年' '否' '好']]
